## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
from keras import regularizers
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
tf.test.is_built_with_cuda()

True

In [3]:
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

True

In [41]:
import sys
import numpy as np
import tensorflow as tf
from datetime import datetime
 
device_name="/gpu:0"
 
shape=(int(10000),int(10000))
 
with tf.device(device_name):
    #形状为shap,元素服从minval和maxval之间的均匀分布
    random_matrix = tf.random_uniform(shape=shape, minval=0, maxval=1)
    dot_operation = tf.matmul(random_matrix, tf.transpose(random_matrix))
    sum_operation = tf.reduce_sum(dot_operation)
startTime = datetime.now()

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:
    result = session.run(sum_operation)
    print(result)
    print("\n" * 2)
print("Shape:", shape, "Device:", device_name)
print("Time taken:", datetime.now() - startTime)

250015840000.0



Shape: (10000, 10000) Device: /gpu:0
Time taken: 0:00:21.331618


In [4]:
import time

def DeviceTest(device_name,size):
    with tf.device(device_name):       
        M1=tf.random_normal([size,size],name='M1')
        M2=tf.random_normal([size,size],name='M2')
        mul=tf.matmul(M1,M2,name='mul')
        sum_result=tf.reduce_sum(mul,name='sum_result')
        

        start_time=time.time()
        
        with tf.Session() as sess:
            result=sess.run(sum_result)
        
        take_time=time.time()-start_time       
        print(device_name,"  size=",size, " Time:",take_time)
        
        return take_time  #這樣append到SET才有東西
    
Gpu_Set=[];  Cpu_set=[];  Size_set=[]

for i in range(0,10001,500):
    TestGpu=DeviceTest("/gpu:0",i)
    TestCpu=DeviceTest("/cpu:0",i)
    
    print("-"*100)
    
    Gpu_Set.append(TestGpu)
    Cpu_set.append(TestCpu)
    Size_set.append(i)

/gpu:0   size= 0  Time: 0.07183051109313965
/cpu:0   size= 0  Time: 0.05189013481140137
----------------------------------------------------------------------------------------------------
/gpu:0   size= 500  Time: 0.05241537094116211
/cpu:0   size= 500  Time: 0.054853200912475586
----------------------------------------------------------------------------------------------------
/gpu:0   size= 1000  Time: 0.05285811424255371
/cpu:0   size= 1000  Time: 0.07083678245544434
----------------------------------------------------------------------------------------------------
/gpu:0   size= 1500  Time: 0.06045389175415039
/cpu:0   size= 1500  Time: 0.09005546569824219
----------------------------------------------------------------------------------------------------
/gpu:0   size= 2000  Time: 0.06182980537414551
/cpu:0   size= 2000  Time: 0.11666488647460938
----------------------------------------------------------------------------------------------------
/gpu:0   size= 2500  Time: 0.056

KeyboardInterrupt: 

In [4]:
'''
This tutorial requires your machine to have 2 GPUs
"/cpu:0": The CPU of your machine.
"/gpu:0": The first GPU of your machine
"/gpu:1": The second GPU of your machine
'''
 
import numpy as np
import tensorflow as tf
import datetime
 
# Processing Units logs
log_device_placement = True
 
# Num of multiplications to perform
n = 10
 
'''
Example: compute A^n + B^n on 2 GPUs
Results on  2 RTX-:
 * Single GPU computation time: 0:00:??
 * Multi GPU computation time: 0:00:??
'''
# Create random large matrix
A = np.random.rand(10000, 10000).astype('float32')
B = np.random.rand(10000, 10000).astype('float32')
 
# Create a graph to store results
c1 = []
c2 = []
 
def matpow(M, n):
    if n < 1: #Abstract cases where n < 1
        return M
    else:
        return tf.matmul(M, matpow(M, n-1))

'''
Single GPU computing
'''
with tf.device('/gpu:0'):
    a = tf.placeholder(tf.float32, [10000, 10000])
    b = tf.placeholder(tf.float32, [10000, 10000])
    # Compute A^n and B^n and store results in c1
    c1.append(matpow(a, n))
    c1.append(matpow(b, n))

with tf.device('/cpu:0'):
    sum = tf.add_n(c1) #Addition of all elements in c1, i.e. A^n + B^n

t1_1 = datetime.datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=log_device_placement)) as sess:
    # Run the op.
    sess.run(sum, {a:A, b:B})
t2_1 = datetime.datetime.now()
 

'''
Multi GPU computing
'''
# GPU:0 computes A^n
with tf.device('/gpu:0'):
    # Compute A^n and store result in c2
    a = tf.placeholder(tf.float32, [10000, 10000])
    c2.append(matpow(a, n))

# GPU:1 computes B^n
#with tf.device('/gpu:1'):
    # Compute B^n and store result in c2
#    b = tf.placeholder(tf.float32, [10000, 10000])
#    c2.append(matpow(b, n))

with tf.device('/cpu:0'):
    sum = tf.add_n(c2) #Addition of all elements in c2, i.e. A^n + B^n

t1_2 = datetime.datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=log_device_placement)) as sess:
    # Run the op.
    sess.run(sum, {a:A, b:B})
t2_2 = datetime.datetime.now()
 
print("Single GPU computation time: " + str(t2_1-t1_1))
print("Multi GPU computation time: " + str(t2_2-t1_2))

Single GPU computation time: 0:00:05.560438
Multi GPU computation time: 0:00:02.501978


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
    mean = np.mean(X_train,axis=(0,1,2,3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train-mean)/(std+1e-7)
    X_test = (X_test-mean)/(std+1e-7) 
    return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [9]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(64, (3,3), padding='same',input_shape=(32,32,3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(BatchNormalization())

#卷積組合
classifier.add(Convolution2D(32, (3,3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100,activation='relu',kernel_regularizer=regularizers.l2(l=0.001)))
classifier.add(BatchNormalization())
classifier.add(Dropout(p=0.01))

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`
C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.01)`
C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 7s 136us/step - loss: 1.4092 - acc: 0.5687
Epoch 2/100
50000/50000 [==============================] - 6s 118us/step - loss: 1.0340 - acc: 0.6912
Epoch 3/100
50000/50000 [==============================] - 6s 117us/step - loss: 0.8876 - acc: 0.7391
Epoch 4/100
50000/50000 [==============================] - 6s 117us/step - loss: 0.8047 - acc: 0.7668
Epoch 5/100
50000/50000 [==============================] - 6s 118us/step - loss: 0.7411 - acc: 0.7906
Epoch 6/100
50000/50000 [==============================] - 6s 118us/step - loss: 0.6948 - acc: 0.8108
Epoch 7/100
50000/50000 [==============================] - 6s 117us/step - loss: 0.6543 - acc: 0.8307
Epoch 8/100
50000/50000 [==============================] - 6s 118us/step - loss: 0.6222 - acc: 0.8435
Epoch 9/100
50000/50000 [==============================] - 6s 119us/step - loss: 0.5892 - acc: 0.8591
Epoch 10/100
50000/50000 [==============================] - 6s 119us/step - loss: 

50000/50000 [==============================] - 6s 119us/step - loss: 0.3985 - acc: 0.9504
Epoch 81/100
50000/50000 [==============================] - 6s 118us/step - loss: 0.3882 - acc: 0.9529
Epoch 82/100
50000/50000 [==============================] - 6s 118us/step - loss: 0.3920 - acc: 0.9509
Epoch 83/100
50000/50000 [==============================] - 6s 118us/step - loss: 0.3921 - acc: 0.9500
Epoch 84/100
50000/50000 [==============================] - 6s 118us/step - loss: 0.4013 - acc: 0.9476
Epoch 85/100
50000/50000 [==============================] - 6s 118us/step - loss: 0.3933 - acc: 0.9513
Epoch 86/100
50000/50000 [==============================] - 6s 119us/step - loss: 0.3905 - acc: 0.9517
Epoch 87/100
50000/50000 [==============================] - 6s 119us/step - loss: 0.3930 - acc: 0.9508
Epoch 88/100
50000/50000 [==============================] - 6s 119us/step - loss: 0.3922 - acc: 0.9509
Epoch 89/100
50000/50000 [==============================] - 6s 117us/step - loss: 0.38

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [10]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[8.3707012e-02, 9.4767461e-07, 2.0855308e-01, 4.2585623e-01,
        1.3898586e-01, 3.5427327e-06, 2.1875611e-02, 8.8977540e-06,
        1.2100813e-01, 7.0388199e-07]], dtype=float32)

In [11]:
score = classifier.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.5770905710220338
Test accuracy: 0.7321


In [15]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3, 3), padding='same',input_shape=(32,32,3), activation='relu'))
classifier.add(Convolution2D(32, (3, 3), padding='same', activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(BatchNormalization())

#卷積組合
classifier.add(Convolution2D(64, (3, 3), padding='same', activation='relu'))
classifier.add(Convolution2D(64, (3, 3), padding='same', activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100,activation='relu',kernel_regularizer=regularizers.l2(0.01)))
classifier.add(BatchNormalization()) 
classifier.add(Dropout(p=0.5))

classifier.add(Dense(output_dim=100,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
classifier.add(BatchNormalization()) 
classifier.add(Dropout(p=0.1))

#輸出
classifier.add(Dense(output_dim=10,activation= 'softmax'))
classifier.summary()

C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`
C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.5)`
C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 8, 64)          0         
__________

C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


In [16]:
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Epoch 1/100
50000/50000 [==============================] - 9s 190us/step - loss: 2.4190 - acc: 0.4660
Epoch 2/100
50000/50000 [==============================] - 8s 164us/step - loss: 1.4523 - acc: 0.6396
Epoch 3/100
50000/50000 [==============================] - 8s 161us/step - loss: 1.3120 - acc: 0.6956
Epoch 4/100
50000/50000 [==============================] - 8s 161us/step - loss: 1.2201 - acc: 0.7267
Epoch 5/100
50000/50000 [==============================] - 8s 162us/step - loss: 1.1703 - acc: 0.7466
Epoch 6/100
50000/50000 [==============================] - 8s 164us/step - loss: 1.1256 - acc: 0.7648
Epoch 7/100
50000/50000 [==============================] - 8s 164us/step - loss: 1.0866 - acc: 0.7767
Epoch 8/100
50000/50000 [==============================] - 8s 164us/step - loss: 1.0612 - acc: 0.7872
Epoch 9/100
50000/50000 [==============================] - 8s 163us/step - loss: 1.0344 - acc: 0.7948
Epoch 10/100
50000/50000 [==============================] - 8s 163us/step - loss: 

50000/50000 [==============================] - 8s 164us/step - loss: 0.6464 - acc: 0.9381
Epoch 81/100
50000/50000 [==============================] - 8s 165us/step - loss: 0.6418 - acc: 0.9378
Epoch 82/100
50000/50000 [==============================] - 8s 166us/step - loss: 0.6430 - acc: 0.9377
Epoch 83/100
50000/50000 [==============================] - 8s 164us/step - loss: 0.6398 - acc: 0.9382
Epoch 84/100
50000/50000 [==============================] - 8s 164us/step - loss: 0.6404 - acc: 0.9382
Epoch 85/100
50000/50000 [==============================] - 8s 166us/step - loss: 0.6378 - acc: 0.9389
Epoch 86/100
50000/50000 [==============================] - 8s 166us/step - loss: 0.6279 - acc: 0.9404
Epoch 87/100
50000/50000 [==============================] - 8s 166us/step - loss: 0.6480 - acc: 0.9380
Epoch 88/100
50000/50000 [==============================] - 8s 167us/step - loss: 0.6268 - acc: 0.9410
Epoch 89/100
50000/50000 [==============================] - 8s 165us/step - loss: 0.62

In [17]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[0.27999887, 0.00196995, 0.05890363, 0.27287757, 0.20220275,
        0.00490784, 0.02788561, 0.00643734, 0.13682283, 0.00799367]],
      dtype=float32)

In [18]:
score = classifier.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.3867085874557494
Test accuracy: 0.785


In [20]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3, 3), padding='same',input_shape=(32,32,3), activation='relu'))
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(BatchNormalization())

#卷積組合
classifier.add(Convolution2D(64, (3, 3), padding='same', activation='relu'))
classifier.add(Convolution2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(BatchNormalization())

#卷積組合
classifier.add(Convolution2D(128, (3, 3), padding='same', activation='relu'))
classifier.add(Convolution2D(128, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=128,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
classifier.add(BatchNormalization()) 
classifier.add(Dropout(p=0.5))

classifier.add(Dense(output_dim=256,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
classifier.add(BatchNormalization()) 
classifier.add(Dropout(p=0.3))

classifier.add(Dense(output_dim=128,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
classifier.add(BatchNormalization()) 
classifier.add(Dropout(p=0.5))

#輸出
classifier.add(Dense(output_dim=10,activation= 'softmax'))
classifier.summary()

C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=128)`
C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.5)`
C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=256)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 15, 15, 32)        128       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 6, 6, 64)          0         
__________

C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:31: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.3)`
C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=128)`
C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.5)`
C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:38: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


In [21]:
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Epoch 1/100
50000/50000 [==============================] - 12s 246us/step - loss: 2.4338 - acc: 0.3451
Epoch 2/100
50000/50000 [==============================] - 10s 200us/step - loss: 1.6880 - acc: 0.5467
Epoch 3/100
50000/50000 [==============================] - 10s 200us/step - loss: 1.3268 - acc: 0.6458
Epoch 4/100
50000/50000 [==============================] - 10s 200us/step - loss: 1.0753 - acc: 0.7106
Epoch 5/100
50000/50000 [==============================] - 10s 201us/step - loss: 0.9139 - acc: 0.7496
Epoch 6/100
50000/50000 [==============================] - 10s 201us/step - loss: 0.8029 - acc: 0.7781
Epoch 7/100
50000/50000 [==============================] - 10s 201us/step - loss: 0.7124 - acc: 0.8048
Epoch 8/100
50000/50000 [==============================] - 10s 201us/step - loss: 0.6446 - acc: 0.8251
Epoch 9/100
50000/50000 [==============================] - 10s 200us/step - loss: 0.5974 - acc: 0.8401
Epoch 10/100
50000/50000 [==============================] - 10s 198us/ste

50000/50000 [==============================] - 10s 196us/step - loss: 0.1231 - acc: 0.9851
Epoch 80/100
50000/50000 [==============================] - 10s 196us/step - loss: 0.1206 - acc: 0.9852
Epoch 81/100
50000/50000 [==============================] - 10s 195us/step - loss: 0.1192 - acc: 0.9861
Epoch 82/100
50000/50000 [==============================] - 10s 194us/step - loss: 0.1137 - acc: 0.9862
Epoch 83/100
50000/50000 [==============================] - 10s 196us/step - loss: 0.1289 - acc: 0.9829
Epoch 84/100
50000/50000 [==============================] - 10s 200us/step - loss: 0.1223 - acc: 0.9855
Epoch 85/100
50000/50000 [==============================] - 10s 197us/step - loss: 0.1174 - acc: 0.9862
Epoch 86/100
50000/50000 [==============================] - 10s 196us/step - loss: 0.1184 - acc: 0.9855
Epoch 87/100
50000/50000 [==============================] - 10s 200us/step - loss: 0.1187 - acc: 0.9862
Epoch 88/100
50000/50000 [==============================] - 10s 200us/step - 

In [22]:
score = classifier.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.0907388064861299
Test accuracy: 0.8112
